In [1]:
from dotenv import load_dotenv
import os

from data_utils import get_followers_from_lists, DATA_PATH, get_scaraped_profiles, get_repliers, get_author_ids
from twitter_utils import get_client, get_profiles

load_dotenv()  # take environment variables from .env.

True

In [3]:
client = get_client()

In [4]:
ids = get_followers_from_lists()

In [5]:
len(ids)

12439182

In [6]:
out_path = os.path.join(DATA_PATH, 'followers_rehydrated.jsonl')
err_path = os.path.join(DATA_PATH, 'followers_missing.jsonl')
already_scraped = get_scaraped_profiles(out_path, err_path)
print(f"{len(already_scraped)} users were already scraped")

ids_ = [id_ for id_ in ids if id_ not in already_scraped]
print(f"{len(ids_)} users left to scrape")

12439182 users were already scraped
0 users left to scrape


In [45]:
errors = get_profiles(ids=ids_, client=client, out_path=out_path, error_path=err_path)

batch: 100%|██████████| 6/6 [00:02<00:00,  2.58it/s]


In [46]:
print(len(errors))

0


In [33]:
other_errors = get_profiles(ids=errors, client=client, out_path=out_path, error_path=err_path)

batch: 100%|██████████| 2/2 [00:00<00:00,  3.16it/s]


In [10]:
repliers = get_repliers()

8157

In [11]:
out_path = os.path.join(DATA_PATH, 'repliers_rehydrated.jsonl')
err_path = os.path.join(DATA_PATH, 'repliers_missing.jsonl')

already_scraped = get_scaraped_profiles(out_path, err_path)
print(f"{len(already_scraped)} users were already scraped")

repliers = [id_ for id_ in repliers if id_ not in already_scraped]
print(f"{len(ids_)} users left to scrape")

follower_errors = get_profiles(ids=repliers, client=client, out_path=out_path, error_path=err_path)

batch: 100%|██████████| 82/82 [00:30<00:00,  2.65it/s]


In [13]:
len(follower_errors)

0

In [15]:
author_ids = set(get_author_ids(2016))
author_ids.update(get_author_ids(2020))
author_ids = list(sorted(author_ids))
print(len(author_ids))

1024


In [16]:
out_path = os.path.join(DATA_PATH, 'authors_rehydrated.jsonl')
err_path = os.path.join(DATA_PATH, 'author_profiles_missing.jsonl')

already_scraped = get_scaraped_profiles(out_path, err_path)
print(f"{len(already_scraped)} users were already scraped")

author_ids = [id_ for id_ in author_ids if id_ not in already_scraped]
print(f"{len(ids_)} users left to scrape")

author_errors = get_profiles(ids=author_ids, client=client, out_path=out_path, error_path=err_path)

batch: 100%|██████████| 11/11 [00:04<00:00,  2.51it/s]


In [17]:
len(author_errors)

0